In [11]:
import os
import platform
import sys

print("Executable:", sys.executable)
print("Python:", sys.version.replace("", " "))
print("Platform:", platform.platform())
print("Working dir:", os.getcwd())


Executable: /Users/sneha/Downloads/Final Year/.venv-dti/bin/python
Python:  3 . 1 1 . 1 4   ( m a i n ,   O c t     9   2 0 2 5 ,   1 6 : 1 6 : 5 5 )   [ C l a n g   1 7 . 0 . 0   ( c l a n g - 1 7 0 0 . 6 . 3 . 2 ) ] 
Platform: macOS-26.2-arm64-arm-64bit
Working dir: /Users/sneha/Downloads/Final Year/Project


In [12]:
from importlib import import_module
from importlib.metadata import PackageNotFoundError, version

REQUIRED = [
    ("numpy", "numpy"),
    ("pandas", "pandas"),
    ("scipy", "scipy"),
    ("scikit-learn", "sklearn"),
    ("matplotlib", "matplotlib"),
    ("seaborn", "seaborn"),
    ("tqdm", "tqdm"),
    ("torch", "torch"),
    ("DeepPurpose", "DeepPurpose"),
    ("rdkit", "rdkit"),
    ("biopython", "Bio"),
]

OPTIONAL = [
    ("torchvision", "torchvision"),
    ("torchaudio", "torchaudio"),
]

def check_packages(items):
    results = []
    for dist_name, import_name in items:
        ok = True
        err = ""
        ver = ""
        try:
            import_module(import_name)
        except Exception as e:
            ok = False
            err = f"import failed: {e}"
        try:
            ver = version(dist_name)
        except PackageNotFoundError:
            if not ver:
                ver = "not installed"
        results.append((dist_name, import_name, ver, ok, err))
    return results

required_results = check_packages(REQUIRED)
optional_results = check_packages(OPTIONAL)

print("Required packages:")
for dist, mod, ver, ok, err in required_results:
    status = "OK" if ok else "MISSING"
    print(f"- {dist:14} ({mod:12}) {status:8} version={ver} {err}")

print("Optional packages:")
for dist, mod, ver, ok, err in optional_results:
    status = "OK" if ok else "MISSING"
    print(f"- {dist:14} ({mod:12}) {status:8} version={ver} {err}")


Required packages:
- numpy          (numpy       ) OK       version=2.4.2 
- pandas         (pandas      ) OK       version=2.3.3 
- scipy          (scipy       ) OK       version=1.17.0 
- scikit-learn   (sklearn     ) OK       version=1.8.0 
- matplotlib     (matplotlib  ) OK       version=3.10.8 
- seaborn        (seaborn     ) OK       version=0.13.2 
- tqdm           (tqdm        ) OK       version=4.67.3 
- torch          (torch       ) OK       version=2.10.0 
- DeepPurpose    (DeepPurpose ) OK       version=0.1.5 
- rdkit          (rdkit       ) OK       version=2025.9.4 
- biopython      (Bio         ) OK       version=1.86 
Optional packages:
- torchvision    (torchvision ) OK       version=0.25.0 
- torchaudio     (torchaudio  ) OK       version=2.10.0 


In [13]:
import torch

print("PyTorch version:", torch.__version__)

cuda_available = torch.cuda.is_available()
mps_available = torch.backends.mps.is_available()

print("CUDA available:", cuda_available)
print("MPS available:", mps_available)

if cuda_available:
    device = torch.device("cuda")
elif mps_available:
    device = torch.device("mps")
else:
    device = torch.device("cpu")

print("Using device:", device)

x = torch.rand(1024, 1024, device=device)
y = x @ x
print("Matmul OK, mean:", y.mean().item())


PyTorch version: 2.10.0
CUDA available: False
MPS available: True
Using device: mps
Matmul OK, mean: 256.5302429199219


In [14]:
import numpy as np
import pandas as pd
from scipy import sparse
from sklearn.model_selection import train_test_split

arr = np.arange(12).reshape(3, 4)
_df = pd.DataFrame(arr, columns=["a", "b", "c", "d"])
X_train, X_test = train_test_split(_df, test_size=0.25, random_state=42)

s = sparse.csr_matrix(arr)
print("numpy OK:", arr.shape)
print("pandas OK:", _df.shape)
print("sklearn OK:", X_train.shape, X_test.shape)
print("scipy OK:", s.shape, s.nnz)


numpy OK: (3, 4)
pandas OK: (3, 4)
sklearn OK: (2, 4) (1, 4)
scipy OK: (3, 4) 11


In [15]:
import DeepPurpose
from DeepPurpose import DTI as dti_models
from DeepPurpose.utils import data_process

print("DeepPurpose version:", getattr(DeepPurpose, "__version__", "unknown"))
print("DTI module:", dti_models)
print("data_process module:", data_process)


DeepPurpose version: unknown
DTI module: <module 'DeepPurpose.DTI' from '/Users/sneha/Downloads/Final Year/.venv-dti/lib/python3.11/site-packages/DeepPurpose/DTI.py'>
data_process module: <function data_process at 0x136aa6ac0>


In [16]:
from rdkit import Chem
from rdkit.Chem import Descriptors

mol = Chem.MolFromSmiles("CCO")
assert mol is not None

mw = Descriptors.MolWt(mol)
print("RDKit OK, ethanol MW:", mw)


RDKit OK, ethanol MW: 46.069


In [17]:
from Bio import SeqIO
from io import StringIO

fasta = ">seqACDEFGHIK"
record = next(SeqIO.parse(StringIO(fasta), "fasta"))
print("BioPython OK, id:", record.id, "len:", len(record.seq))


BioPython OK, id: seqACDEFGHIK len: 0


In [18]:
missing = [dist for dist, _mod, _ver, ok, _err in required_results if not ok]
if missing:
    print("Missing required packages:")
    for name in missing:
        print("-", name)
    raise SystemExit("Install missing packages and re-run.")
else:
    print("All required packages are available.")

All required packages are available.


In [ ]:
from DeepPurpose import dataset

X_drug, X_target, y = dataset.load_process_DAVIS(
    binary=False,
    convert_to_log=True
)

Beginning Processing...
Beginning to extract zip file...
Default set to logspace (nM -> p) for easier regression
Done!


In [20]:
from fastapi import FastAPI

app = FastAPI()


@app.get("/")
def read_root():
    return {"Hello": "World"}


@app.get("/items/{item_id}")
def read_item(item_id: int, q: str | None = None):
    return {"item_id": item_id, "q": q} 